![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

# Callysto's Weekly Data Visualization


## World Children's Day

### Recommended Grade levels: 

### Instructions

Click "Cell" and select "Run All".

This will import the data and run all the code, so you can see this week's data visualization. Scroll back to the top after you’ve run the cells.

![instructions](https://github.com/callysto/data-viz-of-the-week/blob/main/images/instructions.png?raw=true)

**You don't need to do any coding to view the visualizations**.

The plots generated in this notebook are interactive. You can hover over and click on elements to see more information. 

Email contact@callysto.ca if you experience issues.

### About this Notebook

Callysto's Weekly Data Visualization is a learning resource that aims to develop data literacy skills. We provide Grades 5-12 teachers and students with a data visualization, like a graph, to interpret. This companion resource walks learners through how the data visualization is created and interpreted by a data scientist. 

The steps of the data analysis process are listed below and applied to each weekly topic.

1. Question - What are we trying to answer?
2. Gather - Find the data source(s) you will need. 
3. Organize - Arrange the data, so that you can easily explore it. 
4. Explore - Examine the data to look for evidence to answer the question. This includes creating visualizations. 
5. Interpret - Describe what's happening in the data visualization. 
6. Communicate - Explain how the evidence answers the question. 

## Question



### Goal

### Background



## Gather

### Code: 

Run the code cells below to import the libraries we need for this project. Libraries are pre-made code that make it easier to analyze our data.

In [ ]:
import pandas as pd
import plotly.express as px
try:
    import pycountry_convert as pc
except:
    !pip install pycountry_convert
    import pycountry_convert as pc
import ipywidgets as widgets
from ipywidgets import interact
print("Libaries imported")

In [ ]:
percentage_pop = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/world-children's-day/percentageofpopchildren.csv", skiprows=4)
primary_school = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/world-children's-day/outofprimaryschool.csv", skiprows=4)
del percentage_pop['Unnamed: 67']
del primary_school['Unnamed: 67']
display(percentage_pop, primary_school)

In [ ]:
def country_code_to_continent(country_code):
    mapping = pc.map_country_alpha3_to_country_name()
    try:
        country_name = mapping[country_code]
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name
    except:
        return None

print(country_code_to_continent('ABW'))

In [ ]:
country_codes = list(percentage_pop['Country Code'])
print(country_codes)

In [ ]:
for country in country_codes:
    try:
        percentage_pop.loc[percentage_pop['Country Code'] == country, 'Continent'] = country_code_to_continent(country)
    except:
        pass
percentage_pop

In [ ]:
pd.set_option('display.max_rows', None)
countries_without_label = percentage_pop[percentage_pop['Continent'].isna()].reset_index(drop=True)
temp = list(countries_without_label['Country Name'])
display(temp)
pd.reset_option('display.max_rows')

In [ ]:
world_df = percentage_pop[percentage_pop['Country Name'] == 'World']
world_df

In [ ]:
percentage_pop = percentage_pop.dropna(subset=['Continent'])
percentage_pop

In [ ]:
cols_to_check = percentage_pop.columns[4:-1]
hashmap_countries_max = {}
for column in cols_to_check:
    max_val = percentage_pop[column].max()
    results = percentage_pop.loc[percentage_pop[column] == max_val]
    
    country_name = results['Country Name'].values[0]
    hashmap_countries_max[country_name] = 1+hashmap_countries_max.get(country_name, 0)
    
    print(f"{country_name} - {column}: ({max_val}%)")

In [ ]:
cols_to_check = percentage_pop.columns[4:-1]
hashmap_countries_min = {}
for column in cols_to_check:
    min_val = percentage_pop[column].min()
    results = percentage_pop.loc[percentage_pop[column] == min_val]
    
    country_name = results['Country Name'].values[0]
    hashmap_countries_min[country_name] = 1+hashmap_countries_min.get(country_name, 0)
    
    print(f"{country_name} - {column}: ({min_val}%)")

In [ ]:
hashmap_countries_max = {k: v for k, v in sorted(hashmap_countries_max.items(), key=lambda item: item[1])}

print(f"Total number of unique countries for max value: {len(hashmap_countries_max)}")
for country in hashmap_countries_max:
    print(f"{country}, Total count: {hashmap_countries_max[country]}")

print('\n')
hashmap_countries_min = {k: v for k, v in sorted(hashmap_countries_min.items(), key=lambda item: item[1])}

print(f"Total number of unique countries for min val: {len(hashmap_countries_min)}")
for country in hashmap_countries_min:
    print(f"{country}, Total count: {hashmap_countries_min[country]}")

In [ ]:
columns_to_plot = percentage_pop.columns[4:-1]

continents = ['North America', 'Asia', 'Europe', 'Oceania', 'Africa', 'South America']

continent_dropdown = widgets.Dropdown(options=continents, description='Continent')

def update_plot(continent):
    continent_filtered = percentage_pop[percentage_pop['Continent'] == continent]

    per_year_df = pd.melt(continent_filtered, id_vars=['Country Name'], value_vars=columns_to_plot,var_name='Year', value_name='Percentage')
    continental_fig = px.line(per_year_df, x='Year', y='Percentage',color='Country Name', line_group='Country Name', hover_name='Country Name')

    continental_fig.update_layout(title=f'Population Percentage Under 14 in {continent}',xaxis_title='Year',yaxis_title='Percentage',legend_title='Country').show()

interact(update_plot, continent=continent_dropdown)